In [ ]:
import pygrc as gr
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
galaxy = [
"NGC4217",
"NGC5055",
"NGC5585",
"NGC6015",
"NGC6503",
"NGC7331",
]

In [ ]:
df_all = []
for g in galaxy:
    df = gr.Reader.read(filepath="../notebooks/data/"+g+"_rotmod.dat")
    df_all.append(df)
    gr.Plot().overlap(df,"Rad",["Vobs","Vgas","Vbul","Vdisk"],"Distance (kpc)","Velocity",g)

In [ ]:
r = np.linspace(0.01,50,2000)
def mass(r, M0, R0):
#def mass(r, M0, rc, R0,beta):
    #return M0*(np.sqrt(R0/rc)*(r/(r+rc)))**(3*beta)
    return M0*(1- (1+(r/R0))*np.exp(-r/R0))

def mond(r, M0, rc, R0,b, beta):
    a = 1.2e-10
    G = 4.300e-6 #parsecs 
    m = mass(r,M0, R0)
    f = (G*m/r)*(1 + b*(1+(r/rc)))#*10e-3
    return np.sqrt(f)*10e-3

def newton(r, M0, rc, R0,beta):
    a = 1.2e-10
    G = 4.30e-6 #parsecs 
    m = mass(r,M0, R0)
    f = G*m/r
    #f = (G*m/r)*(1/np.sqrt(2)) * np.sqrt(1 + np.sqrt(1 + (r**4) * (2*a/(G*m))**2))
    return np.sqrt(f)*10e-3

In [ ]:
for df in df_all:
    print(df['Rad'].max())
    print(df['Rad'].min())

In [ ]:
m_mond=[]
m_newton=[]

for df in df_all:
    m_mond.append(gr.Fit(df['Rad'],df['Vobs'],1.,1.,3,.35,1.))
    m_newton.append(gr.Fit(df['Rad'],df['Vobs'],1.,1.,3,1.))

In [ ]:
fit_mond = [0]*len(m_mond)
fit_newton = [0]*len(m_newton)
for i in range(len(m_mond)):
    fit_mond[i] = m_mond[i].fit_lsq(mond, [(1.,1e14),(1.,10.),(2.,5.),(0.1,2),(0.1,2)],df_all[i]['errV'],[False,False,True,True,True])
    fit_newton[i] = m_newton[i].fit_lsq(newton, [(1.,1e14),(1.,10.),(1,10),(0.1,2)],df_all[i]['errV'],[False,True,True,True])

In [ ]:
for i in range(len(m_mond)):
    fig, ax =plt.subplots()
    gr.Plot().plot_grc(df_all[i],fit_mond[i],mond,'MOND',galaxy[i],ax)
    gr.Plot().plot_grc(df_all[i],fit_newton[i],newton,'Newton',galaxy[i],ax)
    plt.savefig(galaxy[i]+'_rad_vars_fit.pdf')

In [ ]:
fit_mond[1].draw_mnprofile('M0', band=True)

In [ ]:
fit_mond[4].draw_mncontour('rc', 'M0',cl=[.67,.9,.95])